In [21]:
## data generation
import sys, os
import matplotlib.pyplot as plt
sys.path.insert(0, os.getcwd())



In [19]:
pip install optax==0.1.7


  Attempting uninstall: optax
    Found existing installation: optax 0.1.8
    Uninstalling optax-0.1.8:
      Successfully uninstalled optax-0.1.8
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 3.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
!python3 -m venv .venv
!source .venv/bin/activate      # (Windows PowerShell: .\.venv\Scripts\Activate.ps1)
!pip install numpy


In [6]:
pip install -U jax jaxlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for jax: filename=jax-0.4.13-py3-none-any.whl size=1518704 sha256=dbbf3d3b2dbd69848959967f299880ebf4e58e8b674b4b5c360ba5b778de3c51
  Stored in directory: /Users/banshihan/Library/Caches/pip/wheels/46/d9/15/d2800d4089dc4c77299ac7513c6aa1036f5491edbd2bf6ba16
Successfully built jax
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install flax


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 2.9 MB/s eta 0:00:00a 0:00:01m
  Created wheel for msgpack: filename=msgpack-1.1.0-cp38-cp38-macosx_11_0_arm64.whl size=83182 sha256=9112e7a38080ae0a7a091a1b5f45fd30e3db31545714ee284be90eb489cdab0a
  Stored in directory: /Users/banshihan/Library/Caches/pip/wheels/55/aa/93/797450f0b3d3ac6906a2a32306efbb304940a5a8eb5bdff767
Successfully built msgpack
Note: you may need to restart the kernel to use updated packages.


## Linear Case

In [17]:
from datasets import LinearGaussianDataset, SigmoidDataset, SphereDataset
import numpy as np

# 设置随机种子
seed = 42

# 创建数据集实例
dataset = LinearGaussianDataset(seed=seed, dimension=2, padding_dimension=1)

# 生成数据批次
batch_size = 1000
data = dataset.get_batch(batch_size)  ##:contentReference[oaicite:22]{index=22}




## Linear case data preparation

In [ ]:
!pwd  # 看你当前在哪
!ls   # 确保能看到 seed_linpadding_expts.sh 和 run.py


/Users/banshihan/Documents/GitHub/homework-3-ShihanBan/Diffusion-model-low-dimensional-data
__init__.py              diffusion_utils.py       seed_linpadding_expts.sh
__pycache__              generated_data           training.py
data_generation.ipynb    model.py                 utils.py
datasets.py              networks.py
diffusion.py             run.py


In [18]:
# cell 1
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

# 你的项目代码路径，如在上级目录
import sys
sys.path.append('..')  

from diffusion_model import UNet, GaussianDiffusion
from datasets import SphereDataset, LinearGaussianDataset, SigmoidDataset
from utils import manifold_error, save_samples


TypeError: 'type' object is not subscriptable